In [ ]:
import os
import cv2
import random
import shutil
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras as keras
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from PIL import Image

In [ ]:
from sklearn.preprocessing import LabelEncoder
import numpy as np


train_dir = '/kaggle/input/dogs-cats-images/dataset/training_set'
test_dir = '/kaggle/input/dogs-cats-images/dataset/test_set'

classes = os.listdir(train_dir)

resize_size = 256
crop_size = 224

def preprocess_image(image):
    image = tf.image.resize(image, [resize_size, resize_size], method=tf.image.ResizeMethod.BILINEAR)
    image = tf.image.central_crop(image, central_fraction=crop_size / resize_size)
    image = tf.math.divide(image, 255.0)
    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]
    image = (image - mean) / std

    return image

def resize_and_crop(image, resize_size=256, crop_size=224):
    # Resize with bilinear interpolation
    resized_image = tf.image.resize(image, [resize_size, resize_size], method=tf.image.ResizeMethod.BILINEAR)
    cropped_image = tf.image.central_crop(resized_image, central_fraction=crop_size / resize_size)
    image = tf.math.divide(cropped_image, 255.0) #normalize

    return image

def preprocess_for_attack(image):
    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]
    image = (image - mean) / std #다 normalize

    return image

X = []
Original_X = []
y = []
for i in classes:
    for directory in [train_dir, test_dir]:
        folderPath = os.path.join(directory,i)
        for j in tqdm(os.listdir(folderPath)):
            img = cv2.imread(os.path.join(folderPath,j))
            O_img = resize_and_crop(img)
            Original_X.append(O_img)
            img = preprocess_image(img) #전처리
            X.append(img) #X list 넣고
            y.append(i) 
            
#X = np.array(X)
#y = np.array(y)
#y = tf.keras.utils.to_categorical([classes.index(label) for label in y])
#Original_X = np.array(Original_X)

In [ ]:
y = tf.keras.utils.to_categorical([classes.index(label) for label in y])

In [ ]:
from sklearn.metrics import classification_report,accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from tensorflow.keras import layers, models
from vit_keras import vit, utils
from lime.lime_image import LimeImageExplainer
import matplotlib.pyplot as plt
from skimage.segmentation import mark_boundaries
import shap

model_name = "vgg_"
model = keras.models.load_model(f'/kaggle/input/models/{model_name}best_model.h5', compile=False)
model.compile(optimizer='SGD', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
X_train,xx,y_train,yy = train_test_split(X, y, test_size=0.2,random_state=42)
X_val,X_test,y_val,y_test = train_test_split(xx, yy, test_size=0.5,random_state=42)
y_pred = model.predict(np.array(X_test))
y_pred_single_label = np.argmax(y_pred, axis=1)
y_test_single_label = np.argmax(y_test, axis=1)
precision = precision_score(y_test_single_label, y_pred_single_label, average='macro')
recall = recall_score(y_test_single_label, y_pred_single_label, average='macro')
f1 = f1_score(y_test_single_label, y_pred_single_label, average='macro')
accuracy = accuracy_score(y_test_single_label, y_pred_single_label)
precision = round(precision, 3)
recall = round(recall, 3)
f1 = round(f1, 3)
accuracy = round(accuracy, 3)
print(str(model_name))
print(precision, recall, f1, accuracy)

In [ ]:
model_name = "resnet_"
model = keras.models.load_model(f'/kaggle/input/models/{model_name}best_model.h5', compile=False)
model.compile(optimizer='SGD', loss='categorical_crossentropy', metrics=['accuracy'])
y_pred = model.predict(np.array(X_test))
y_pred_single_label = np.argmax(y_pred, axis=1)
y_test_single_label = np.argmax(y_test, axis=1)
precision = precision_score(y_test_single_label, y_pred_single_label, average='macro')
recall = recall_score(y_test_single_label, y_pred_single_label, average='macro')
f1 = f1_score(y_test_single_label, y_pred_single_label, average='macro')
accuracy = accuracy_score(y_test_single_label, y_pred_single_label)
precision = round(precision, 3)
recall = round(recall, 3)
f1 = round(f1, 3)
accuracy = round(accuracy, 3)
print(str(model_name))
print(precision, recall, f1, accuracy)

In [ ]:
model_name = "eff_"
model = keras.models.load_model(f'/kaggle/input/models/{model_name}best_model.h5', compile=False)
model.compile(optimizer='SGD', loss='categorical_crossentropy', metrics=['accuracy'])
y_pred = model.predict(np.array(X_test))
y_pred_single_label = np.argmax(y_pred, axis=1)
y_test_single_label = np.argmax(y_test, axis=1)
precision = precision_score(y_test_single_label, y_pred_single_label, average='macro')
recall = recall_score(y_test_single_label, y_pred_single_label, average='macro')
f1 = f1_score(y_test_single_label, y_pred_single_label, average='macro')
accuracy = accuracy_score(y_test_single_label, y_pred_single_label)
precision = round(precision, 3)
recall = round(recall, 3)
f1 = round(f1, 3)
accuracy = round(accuracy, 3)
print(str(model_name))
print(precision, recall, f1, accuracy)

In [ ]:
model_name = "vit_"
model = keras.models.load_model(f'/kaggle/input/models/{model_name}best_model.h5', compile=False)
model.compile(optimizer='SGD', loss='categorical_crossentropy', metrics=['accuracy'])
y_pred = model.predict(np.array(X_test))
y_pred_single_label = np.argmax(y_pred, axis=1)
y_test_single_label = np.argmax(y_test, axis=1)
precision = precision_score(y_test_single_label, y_pred_single_label, average='macro')
recall = recall_score(y_test_single_label, y_pred_single_label, average='macro')
f1 = f1_score(y_test_single_label, y_pred_single_label, average='macro')
accuracy = accuracy_score(y_test_single_label, y_pred_single_label)
precision = round(precision, 3)
recall = round(recall, 3)
f1 = round(f1, 3)
accuracy = round(accuracy, 3)
print(str(model_name))
print(precision, recall, f1, accuracy)